In [22]:
import os
import json


data_dir = 'envs/hr_payroll/data'

json_data = {}

for filename in os.listdir(data_dir):
    if filename.endswith('.json'):
        file_path = os.path.join(data_dir, filename)
        with open(file_path, 'r') as file:
            key_name = filename.replace('.json', '')  # Remove .json extension
            json_data[key_name] = json.load(file)

print("Imported JSON files:", list(json_data.keys()))

# Access specific data like this:
# json_data['users'], json_data['contracts'], etc.


Imported JSON files: ['bank_accounts', 'contracts', 'documents', 'financial_providers', 'invoices', 'organizations', 'org_departments', 'payments', 'payroll_items', 'payroll_runs', 'reimbursements', 'teams', 'team_members', 'time_entries', 'users', 'virtual_cards', 'workers']


In [ ]:
import os
import ast
import json
from datetime import datetime, timezone
from collections import defaultdict

TOOLS_DIR = "envs/hr_payroll/tools/interface_2"
EXCLUDED_SELECTIONS = {"find_user", "get_contracts", "get_documents", "get_payments"}
SPECIAL_CASE = {"create_new_contract"}

def extract_get_info_dict_from_code(code):
    try:
        tree = ast.parse(code)
        for node in ast.walk(tree):
            if isinstance(node, ast.FunctionDef) and node.name == "get_info":
                for child in ast.walk(node):
                    if isinstance(child, ast.Return):
                        return ast.literal_eval(child.value)
    except Exception as e:
        print(f"Error parsing AST: {e}")
    return {}

def extract_required_fields(parsed_dict):
    try:
        params = parsed_dict.get("function", {}).get("parameters", {})
        required = params.get("required", [])
        if required:
            return required
        return list(params.get("properties", {}).keys())
    except Exception:
        return []
def terms():

    start_date= input("Enter the start date (YYYY-MM-DD): ")
    now_utc = datetime.now(timezone.utc)
    current_time = now_utc.strftime("%H:%M:%S")
    datetime_str = f"{start_date} {current_time}"
    dt = datetime.strptime(datetime_str, "%Y-%m-%d %H:%M:%S")
    dt = dt.replace(tzinfo=timezone.utc)
    formatted_start_date = dt.strftime("%Y-%m-%dT%H:%M:%SZ")
    end_date =input("Enter the end date (YYYY-MM-DD): ")
    now_utc1 = datetime.now(timezone.utc)
    current_time1 = now_utc1.strftime("%H:%M:%S")
    datetime_str1 = f"{end_date} {current_time1}"
    dt1 = datetime.strptime(datetime_str1, "%Y-%m-%d %H:%M:%S")
    dt1 = dt1.replace(tzinfo=timezone.utc)
    formatted_end_date = dt.strftime("%Y-%m-%dT%H:%M:%SZ")
    rate= float(input("Enter the rate amount"))
    rate_type=input("Enter the rate type")
    currency=input("enter the currency")
    return {
            "start_date":formatted_start_date, 
            "end_date":formatted_end_date, 
            "rate":rate, 
            "rate_type":rate_type, 
            "currency":currency
        }

import uuid

def generate_unique_payment_id(existing_ids):
    while True:
        uid = str(uuid.uuid4())
        if uid not in existing_ids:
            return uid

    
def output_manupulation(tool_id,input_cache):
    if tool_id == 1:
        time_entry_id = input_cache.get("time_entry_id") or input("Enter time_entry_id for output: ")
        result = [
            value 
            for entry_id, value in json_data["time_entries"].items()
            if entry_id == time_entry_id
        ]
        if result[0]["status"] in ["submitted", "draft"]:
            result[0]["status"] = "approved"
        return result[0] if result else "time entry not found"
    if tool_id ==2:
        payment_id = input_cache.get("payment_id") or input("Enter the payment id for output:")
        result = [
            value for key, value in json_data["payments"].items() 
            if key == payment_id
        ]
        if result[0]['status']!= "failed":
            result[0]['status'] ="blocked"
        else:
            return f"payment in status {result[0]['status']} cannot be blocked"
        return {"payment_id":payment_id,"status":result[0]['status']} if result else "payment not found"
    
    if tool_id ==3:
        card_id = input_cache.get("card_id") or input("Enter the card id for output")
        result = [
            value for id, value in json_data["virtual_cards"].items() 
            if id == card_id
        ]
        if result[0].get("status") not in ["revoked", "expired", "blocked"]:
            result[0]['status'] ="blocked"
        else:
            return f"card in status {result[0]['status']} cannot be blocked"
        return {"card_id":card_id,"status":result[0]['status']} if result else "card not found"
    
    if tool_id ==4:
        card_id = input_cache.get("card_id") or input("Enter the card id for checking virtual card for output")
        result = [
            value for id, value in json_data["virtual_cards"].items() 
            if id == card_id
        ]
        return {"card_id":card_id,"status":result[0]['status']} if result else "card not found"
    
    if tool_id ==5:
        worker_id = input_cache.get("worker_id") or input("Enter the worker id for output:")
        terms = input_cache.get("terms")
        return {
            "worker_id":worker_id,
            "terms":terms
        }

    if tool_id == 6:
        payment_id = generate_unique_payment_id(json_data['payments'].keys())
        user_id =input_cache.get("user_id") or input("Enter user id for output")
        invoice_id = input_cache.get("invoice_id") or input("Enter invoice id for output")
        amount= input_cache.get("amount") or input("Enter amount for output")
        currency= input_cache.get("currency") or input("Enter currency for output")
        processed_at= input("Enter the processing date (YYYY-MM-DD): ")
        now_utc = datetime.now(timezone.utc)
        current_time = now_utc.strftime("%H:%M:%S")
        datetime_str = f"{processed_at} {current_time}"
        dt = datetime.strptime(datetime_str, "%Y-%m-%d %H:%M:%S")
        dt = dt.replace(tzinfo=timezone.utc)
        formatted_processed_date = dt.strftime("%Y-%m-%dT%H:%M:%SZ")
        return {
        "payment_id": payment_id,
        "invoice_id":invoice_id ,
        "amount": amount,
        "currency": currency,
        "status": "pending",
        "processed_at": formatted_processed_date,
        "user_id": user_id
        }
    if tool_id == 7:
        contract_id = input_cache.get("contract_id") or input("Enter the contract id for extending the contract for output: ")
        new_end_date = input_cache.get("new_end_date") or input("Enter the new end date (YYYY-MM-DD): ")

        # Filter to get the matching contract
        result = [
            value for id, value in json_data["contracts"].items() 
            if id == contract_id
        ]

        # Update the end_date if a matching contract is found
        if result:
            result[0]["end_date"] = new_end_date
            return result[0]
        else:
            return "no contract was made"
    
    if tool_id == 8:
        team_id = input_cache.get("team_id",0)

        worker_ids = [
            value['worker_id']
            for value in json_data['team_members'].values()
            if value["team_id"] == team_id
        ]

        entries = [
                    {
                        'worker_id': value['worker_id'],
                        'duration_hours': value['duration_hours'],
                        'date':value['date']
                    }
                    for value in json_data['time_entries'].values()
                    if value['worker_id'] in worker_ids
                ]

        daily_totals = defaultdict(float)
        for entry in entries:
            daily_totals[entry['date']] += float(entry['duration_hours'])

        result = {
            "team_id": team_id,
            "entries": entries,
            "daily_totals": dict(daily_totals)
        }
        return result
    if tool_id == 9:
        inps = (
            input_cache.get("user_id") or
            input_cache.get("email") or
            input_cache.get("first_name") or
            input_cache.get("last_name") or
            input_cache.get("role") or
            input_cache.get("status") or
            input_cache.get("locale") or
            input_cache.get("timezone")
        )
        user =[]
        # Iterate over both keys (user_id) and values (user info)
        result = [
            {**value, 'user_id': user_id}
            for user_id, value in json_data['users'].items()
            if inps in (
                user_id,
                value.get('email'),
                value.get('first_name'),
                value.get('last_name'),
                value.get('role'),
                value.get('status'),
                value.get('locale'),
                value.get('timezone')
            )
        ]
        worker_details = next(
                        (val for val in json_data["workers"].values() if val.get("user_id") == result[0]["user_id"]),
                        None
                    )

        return user.append({
            "user": result[0],
            "worker": worker_details
        })
    if tool_id ==10:
        inps = (
            input_cache.get("contract_id") or
            input_cache.get("user_id") or
            input_cache.get("worker_id") or
            input_cache.get("status") or
            input_cache.get("currency") or
            input_cache.get("organization_id") or
            input_cache.get("rate_type") or
            input_cache.get("document_id") or
            input_cache.get("min_rate") or
            input_cache.get("max_rate") or
            input_cache.get("start_date_from") or
            input_cache.get("start_date_to") or
            input_cache.get("end_date_from") or
            input_cache.get("end_date_to")
        )
        result = [
            {**value, 'contract_id': contract_id}
            for contract_id, value in json_data['contracts'].items()
            if inps in (
                contract_id,
                value.get('user_id'),
                value.get('worker_id'),
                value.get('status'),
                value.get('currency'),
                value.get('organization_id'),
                value.get('rate_type'),
                value.get('document_id'),
                str(value.get('min_rate')),  # Convert to string in case input is string
                str(value.get('max_rate')),
                value.get('start_date_from'),
                value.get('start_date_to'),
                value.get('end_date_from'),
                value.get('end_date_to')
            )
        ]
        return result[0] if result else "contract details not found for given input"
    if tool_id == 11:
        inps = (
            input_cache.get("document_id") or
            input_cache.get("user_id") or
            input_cache.get("worker_id") or
            input_cache.get("title") or
            input_cache.get("file_type") or
            input_cache.get("status")
        )
        result = [
            {**value, 'document_id': document_id}
            for document_id, value in json_data['documents'].items()
            if inps in (
                document_id,
                value.get('user_id'),
                value.get('worker_id'),
                value.get('title'),
                value.get('file_type'),
                value.get('status')
            )
        ]

        return result[0] if result else "document not found"
    if tool_id ==12:
        # Step 1: Get the first available input from input_cache
        inps = (
            input_cache.get("payment_id") or
            input_cache.get("user_id") or
            input_cache.get("invoice_id") or
            input_cache.get("status") or
            input_cache.get("currency") or
            input_cache.get("processed_at") or
            input_cache.get("min_amount") or
            input_cache.get("max_amount")
        )

        # Step 2: Search through json_data['payments']
        result = [
            {**value, 'payment_id': payment_id}
            for payment_id, value in json_data['payments'].items()
            if inps in (
                payment_id,
                value.get('user_id'),
                value.get('invoice_id'),
                value.get('status'),
                value.get('currency'),
                value.get('processed_at'),
                str(value.get('min_amount')),
                str(value.get('max_amount'))
            )
        ]

        return [res for res in result] if result else f"payment details not found {inps} "
    
    if tool_id == 13:
        payroll_run_id = input_cache.get("payroll_run_id")
        result = [value for id,value in json_data['payroll_runs'].items() if id==payroll_run_id]
        return {"payroll_run_id":payroll_run_id,"items": result[0]}
    
    if tool_id ==14:
        organization_id = input_cache.get("organization_id")
        result = [value for id,value in json_data['reimbursements'].items() if value["organization_id"]==organization_id and value["status"].lower()=="submitted"]
        return result
    
    if tool_id == 15:
        worker_id = input_cache.get("worker_id")
        org_id = input_cache.get("org_id")
        result = [value for id,value in json_data['contracts'].items() if value["worker_id"]==worker_id and value["organization_id"]==org_id]
        if result:
            return result[0]
        else:
            return "no contract found"
    
    if tool_id == 16:
        name = input_cache.get("name")
        org_name = input_cache.get("organization_id")

        # Step 1: Get the organization_id from organization name
        organization_id = None
        for oid, org in json_data["organizations"].items():
            if org.get("name", "").lower() == org_name.lower():
                organization_id = oid
                break

        if not organization_id:
            return "Organization not found"

        # Step 2: Find user_id by full name and organization_id
        matched_user_id = None
        for uid, u in json_data['users'].items():
            full_name = f"{u.get('first_name', '').strip()} {u.get('last_name', '').strip()}".strip()
            if full_name.lower() == name.strip().lower() and u.get("organization_id") == organization_id:
                matched_user_id = uid
                break

        if not matched_user_id:
            return "User not found"

        # Step 3: Return worker entry for matched user_id and organization_id
        for worker_id, worker in json_data['workers'].items():
            if worker.get("user_id") == matched_user_id and worker.get("organization_id") == organization_id:
                return {worker_id: worker}

        return "Worker not found"
        
    if tool_id == 17:
        document_id = generate_unique_payment_id(json_data['documents'].keys())
        new_status = input_cache.get("new_status")
        result = [value for id,value in json_data['documents'].items() if  value["status"] not in ["archived", "deleted"] ]#id==document_id and
        result[0]["status"] = new_status
        return {"docuemnt_id":document_id,"new_status":result[0]["status"]}
    
    if tool_id ==18:
        user_id= input_cache.get("user_id")
        worker_id = input_cache.get("worker_id")
        title = input_cache.get("title")
        file_type = input_cache.get("file_type")
        return {
            "document_id":document_id,
            "user_id": user_id,
            "worker_id": worker_id,
            "title": title,
            "file_type":file_type,
            "status":"pending"
        }

# Load tools
tools = [
    f.replace('.py', '')
    for f in os.listdir(TOOLS_DIR)
    if f.endswith('.py') and f != '__init__.py'
]
tool_dict = {i + 1: tool for i, tool in enumerate(tools)}
    
    

# Show menu
def show_menu():
    print("\n📋 Available Tools:")
    for idx, name in tool_dict.items():
        print(f"{idx}. {name}")
    print("0. Done (exit and print final payload)")

all_data_entries = []
input_cache = {}


while True:
    show_menu()
    try:
        choice = int(input("\n🔢 Enter the number of the tool to inspect (0 to finish): "))
    except ValueError:
        print("❌ Please enter a valid number.")
        continue

    if choice == 0:
        break

    selected_tool = tool_dict.get(choice)
    if not selected_tool:
        print("❌ Invalid selection.")
        continue

    print(f"\n🔍 You selected: {selected_tool}")
    file_path = os.path.join(TOOLS_DIR, f"{selected_tool}.py")

    if not os.path.exists(file_path):
        print(f"❌ Tool file not found: {file_path}")
        continue

    with open(file_path, "r", encoding="utf-8") as f:
        code = f.read()
        info_dict = extract_get_info_dict_from_code(code)
        required_fields = extract_required_fields(info_dict)

    # Custom handling for 9–12 tools
    if selected_tool in EXCLUDED_SELECTIONS:
        print("\n🧮 Choose which fields to include (comma-separated):")
        for idx, field in enumerate(required_fields, 1):
            print(f"  {idx}. {field}")
        try:
            selected_indices = input("➤ Enter field numbers (e.g., 1,3): ")
            indices = {int(i.strip()) for i in selected_indices.split(",") if i.strip().isdigit()}
            required_fields = [field for i, field in enumerate(required_fields, 1) if i in indices]
        except Exception as e:
            print(f"⚠️ Invalid input. Skipping this tool. Error: {e}")
            continue       

    # Collect values (empty for now)
    # for key in required_fields:
    #     if key == "terms":
    #        arguments[key] = terms()
    #     else:
    #       arguments=  {key:input(f"enter{key} for {selected_tool}")}
    arguments = {}
    for key in required_fields:
        if key == "terms":
            arguments[key] = terms()
            input_cache[key] = arguments[key]
        else:
            val = input(f"Enter {key} for {selected_tool}: ")
            arguments[key] = val
            input_cache[key] = val


    # arguments = {key: input(f"enter{key} for {selected_tool}") for key in required_fields}
    output_data = output_manupulation(choice, input_cache)
    # Construct one data block
    data_entry = {
        "name": selected_tool,
        "arguments": arguments,
        "output":[output_data]

    }

    all_data_entries.append(data_entry)

# Final output
print("\n📦 Final Payload:")
print(json.dumps(all_data_entries, indent=4))

# Optional: Save to file
with open("final_payload.json", "w") as f:
    json.dump(all_data_entries, f, indent=4)
    print("✅ Saved to final_payload.json")



📋 Available Tools:
1. approve_overtime_entry
2. block_suspicious_payment
3. block_virtual_card
4. check_user_virtual_cards
5. create_new_contract
6. create_payment
7. extend_contract_period
8. fetch_time_summary_by_team
9. find_user
10. get_contracts
11. get_documents
12. get_payments
13. get_payroll_run_details
14. get_pending_reimbursements
15. retrieve_worker_contracts_with_organization
16. start_new_engagement
17. update_document_status
18. upload_document
0. Done (exit and print final payload)

🔍 You selected: block_suspicious_payment


IndexError: list index out of range